In [2]:
using DataFrames, CSV

In [9]:
fuel_data = CSV.read("Fuels_data.csv", DataFrame)



Row,Fuel,Cost_per_MMBtu,CO2_content_tons_per_MMBtu,fuel_indices
,String31,Float64,Float64,Int64
1,None,0.0,0.0,1
2,ercot_coal,1.91,0.09552,2
3,ercot_naturalgas,3.62,0.05306,3
4,ercot_uranium,0.71,0.0,4
5,ercot_naturalgas_ccs90,4.1,0.00531,5
6,ercot_naturalgas_ccs100,4.15,0.0,6


In [10]:
generator_data = CSV.read("Generators_data.csv", DataFrame)

Row,region,Resource,cluster,R_ID,zone,voltage_level,CapRes,THERM,DISP,NDISP,STOR,DR,HEAT,NACC,TRADER,HYDRO,RPS,CES,Commit,Min_Share,Max_Share,Existing_Cap_MWh,Existing_Cap_MW,num_units,unmodified_existing_cap_mw,New_Build,Cap_size,Min_Cap_MW,Max_Cap_MW,Min_Share_percent,Max_Share_percent,capex,Inv_cost_per_MWyr,Fixed_OM_cost_per_MWyr,capex_mwh,Inv_cost_per_MWhyr,Fixed_OM_cost_per_MWhyr,Var_OM_cost_per_MWh,Var_OM_cost_per_MWh_in,Externality_cost_MWh,Start_cost_per_MW,Start_fuel_MMBTU_per_MW,Heat_rate_MMBTU_per_MWh,heat_rate_mmbtu_mwh_iqr,heat_rate_mmbtu_mwh_std,Fuel,Min_power,Self_disch,Eff_up,Eff_down,Ratio_power_to_energy,Min_Duration,Max_Duration,Max_DSM_delay,Ramp_Up_percentage,Ramp_Dn_percentage,Up_time,Down_time,NACC_Eff,NACC_Peak_to_Base,Reg_Up,Reg_Dn,Rsv_Up,Rsv_Dn,Reg_Cost,Rsv_Cost,spur_miles,spur_line_capex,interconnect_annuity,Power_uprate_cost_per_MWyr,Max_Uprate_MW,Hydro_level,Existing_Charge_Cap_MW,Existing_cap_MWh
,String7,String,String7,Int64,Int64,String7,Float64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,ERC_P,onshore_wind_turbine,1,1,1,trans,0.0,0,1,0,0,0,0.0,0.0,0.0,0,1,1,0,0.0,0.0,0.0,8495.9,22.0,4184.71,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,45062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.12,0.0,0.02,None,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0,1.0,1.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ERC_R,biomass,1,2,2,trans,0.0,0,0,1,0,0,0.0,0.0,0.0,0,1,1,1,0.0,0.0,0.0,48.4,44.0,178.904,-1,1.1,0.0,0.0,0.0,0.0,0.0,0.0,890411,0.0,0.0,0.0,44.4,0.0,0.0,0.0,0.0,103.17,1.104,192.239,None,0.593,0.0,1.0,1.0,1.0,0.0,0.0,0,1.0,1.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ERC_R,conventional_hydroelectric,1,3,2,trans,0.0,0,0,0,0,0,0.0,0.0,0.0,1,0,1,0,0.0,0.0,0.0,467.02,19.0,467.001,-1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,71207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.12,0.0,0.0,None,0.005,0.0,1.0,1.0,1.0,0.0,0.0,0,0.083,0.083,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0
4,ERC_R,conventional_steam_coal,1,4,2,trans,0.0,1,0,0,0,0,0.0,0.0,0.0,0,0,0,1,0.0,0.0,0.0,13567.7,21.0,13567.6,0,646.08,0.0,0.0,0.0,0.0,0.0,0.0,41917,0.0,0.0,0.0,6.2,0.0,0.0,122.0,13.7,10.5,0.822,0.675,ercot_coal,0.356,0.0,1.0,1.0,1.0,0.0,0.0,0,0.57,0.57,24,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,ERC_R,natural_gas_fired_combined_cycle,1,5,2,trans,0.0,1,0,0,0,0,0.0,0.0,0.0,0,0,0,1,0.0,0.0,0.0,36734.2,79.0,36733.9,0,464.99,0.0,0.0,0.0,0.0,0.0,0.0,13642,0.0,0.0,0.0,4.1,0.0,0.0,91.0,2.0,8.05,1.807,2.291,ercot_naturalgas,0.464,0.0,1.0,1.0,1.0,0.0,0.0,0,0.64,0.64,6,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,ERC_R,natural_gas_fired_combustion_turbine,1,6,2,trans,0.0,1,0,0,0,0,0.0,0.0,0.0,0,0,0,1,0.0,0.0,0.0,4704.48,72.0,4704.41,0,65.34,0.0,0.0,0.0,0.0,0.0,0.0,8670,0.0,0.0,0.0,11.7,0.0,0.0,118.0,3.5,10.12,3.367,3.499,ercot_naturalgas,0.542,0.0,1.0,1.0,1.0,0.0,0.0,0,3.78,3.78,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,ERC_R,natural_gas_steam_turbine,1,7,2,trans,0.0,1,0,0,0,0,0.0,0.0,0.0,0,0,0,1,0.0,0.0,0.0,1118.5,5.0,1118.5,0,223.7,0.0,0.0,0.0,0.0,0.0,0.0,47301,0.0,0.0,0.0,7.2,0.0,0.0,86.0,13.7,12.09,0.678,0.611,ercot_naturalgas,0.202,0.0,1.0,1.0,1.0,0.0,0.0,0,3.78,3.78,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,ERC_R,nuclear,1,8,2,trans,0.0,1,0,0,0,0,0.0,0.0,0.0,0,0,1,1,0.0,0.0,0.0,5020.0,4.0,5020.0,-1,1255.0,0.0,0.0,0.0,0.0,0.0,0.0,105165,0.0,0.0,0.0,2.4,0.0,0.0,245.0,0.0,10.46,0.0,0.0,ercot_uranium,0.5,0.0,1.0,1.0,1.0,0.0,0.0,0,0.25,0.25,24,24,0.0,0.0,0.0,0.0,0.0

In [13]:
demand_data = CSV.read("Load_data.csv", DataFrame);
demand_data = select(demand_data, :Time_index, :Load_MW_z1, :Load_MW_z2, :Load_MW_z3);